In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
import numpy.matlib
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel, delayed

## Create a geodataframe representation of the MERRA2 Grid

In [3]:
#load in a merra dataset
dat_merra = nc.Dataset('/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191225.nc4')
lat_merra = dat_merra['lat'][:].data
lon_merra= dat_merra['lon'][:].data

dlon_merra = 0.625/2
dlat_merra = 0.5/2

In [22]:
#make a dataframe of the polygons for the merra dataset

xx = [np.matlib.repmat(np.array(i),len(lon_merra),1)[:,0] for i in range(len(lat_merra))]
yy = [np.matlib.repmat(np.arange(len(lat_merra)),1,len(lon_merra))]

lat_inds = []
for element in xx:
    for item in element:
        lat_inds.append(item)       
print(len(lat_inds))

lon_inds = []
for element in yy:
    for item in element:
        lon_inds.append(item)       
#print(lon_inds[0].shape)
df_merra = gpd.GeoDataFrame({'row':lat_inds, 
                        'col': lon_inds[0], 
                        'geometry': np.zeros(len(lon_merra)*len(lat_merra))})

for ii in range(len(lat_merra)):
    for jj in range(len(lon_merra)):
        poly_cell = Polygon([(lon_merra[jj]-dlon_merra,lat_merra[ii]+dlat_merra), 
                     (lon_merra[jj]+dlon_merra,lat_merra[ii]+dlat_merra), 
                     (lon_merra[jj]+dlon_merra,lat_merra[ii]-dlat_merra), 
                     (lon_merra[jj]-dlon_merra, lat_merra[ii]-dlat_merra)])
        #print(ii*len(lon)+jj)
        df_merra.loc[ii*len(lon_merra)+jj, 'geometry'] = poly_cell
        
print(type(df_merra))

hi = gpd.GeoDataFrame(df_merra['geometry'])
print(type(hi))
therest = pd.DataFrame(df_merra[['row', 'col']])


hi.to_file("merra_grid_boxes_geometry.geojson", driver='GeoJSON')
therest.to_csv("merra_grid_boxes_indices.csv")

1443
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


## Create a GeoDataFrame Representation of the QFED Grid

In [27]:
#load in a merra dataset
dat_qfed = nc.Dataset('/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190901.nc4')
lat_qfed = dat_qfed['lat'][:].data
lon_qfed= dat_qfed['lon'][:].data

dlon_qfed = 0.1/2
dlat_qfed = 0.1/2

In [30]:
#make a dataframe of the polygons for the merra dataset

xx = [np.matlib.repmat(np.array(i),len(lon_qfed),1)[:,0] for i in range(len(lat_qfed))]
yy = [np.matlib.repmat(np.arange(len(lat_qfed)),1,len(lon_qfed))]

lat_inds = []
for element in xx:
    for item in element:
        lat_inds.append(item)       
print(len(lat_inds))

lon_inds = []
for element in yy:
    for item in element:
        lon_inds.append(item)       
#print(lon_inds[0].shape)
df_qfed = gpd.GeoDataFrame({'row':lat_inds, 
                        'col': lon_inds[0], 
                        'geometry': np.zeros(len(lon_qfed)*len(lat_qfed))})

for ii in range(len(lat_qfed)):
    for jj in range(len(lon_qfed)):
        poly_cell = Polygon([(lon_qfed[jj]-dlon_qfed,lat_qfed[ii]+dlat_qfed), 
                     (lon_qfed[jj]+dlon_qfed,lat_qfed[ii]+dlat_qfed), 
                     (lon_qfed[jj]+dlon_qfed,lat_qfed[ii]-dlat_qfed), 
                     (lon_qfed[jj]-dlon_qfed, lat_qfed[ii]-dlat_qfed)])
        #print(ii*len(lon)+jj)
        df_qfed.loc[ii*len(lon_qfed)+jj, 'geometry'] = poly_cell
        
#print(df_qfed)

hi = gpd.GeoDataFrame(df_qfed['geometry'])
print(type(hi))
therest = pd.DataFrame(df_qfed[['row', 'col']])


hi.to_file("qfed_grid_boxes_geometry.geojson", driver='GeoJSON')
therest.to_csv("qfed_grid_boxes_indices.csv")

43200
<class 'geopandas.geodataframe.GeoDataFrame'>


## Do other stuff

In [61]:
#df_grid is a dataframe of grid cells as polygons and the associated cell indices
#fire_shape is the polygon associated with the fire on a given day
def calc_intersection(df_grid, fire_shape):
    df_grid_geometry = gpd.GeoSeries(df_grid['geometry']) 
    areas= df_grid_geometry.intersection(fire_shape, align=False).area #a dataframe of the intersections
    #return a dataframe with the indices and overlaps
    df_return = pd.DataFrame({'row': df_grid[areas!=0]['row'], 'col': df_grid[areas!=0]['col'], 'overlap':areas[areas!=0]/fire_shape.area })
    print(df_return)

In [70]:
calc_intersection(df, fire_polygon)

     row  col  overlap
504   12   23      1.0


In [62]:
fire_polygons = gpd.read_file('fire_polygons_only.geojson')
df_qfed_boxes = gpd.read_file('qfed_grid_boxes_geometry.geojson')
df_qfed_indices = pd.read_csv('qfed_grid_boxes_indices.csv')

df_qfed = pd.concat([df_qfed_indices, df_qfed_boxes], axis=1)
df_qfed.loc[0:55]
print(fire_polygons.iloc[0][0])
for nn in range(1):#len(fire_polygons)):
    tic = time.time()
    print(nn)
    fire_poly = fire_polygons.iloc[nn]
    fire_poly = fire_poly[0]
    calc_intersection(df_qfed, fire_poly)
    toc=time.time()
    print(toc-tic)

POLYGON ((-102.4847222 36.9952778, -102.4849148109331 36.99135711438682, -102.4854907887839 36.98747418711935, -102.4864445865707 36.98366641290982, -102.4877670186995 36.9799704627054, -102.4894453494261 36.97642193052695, -102.4914634155079 36.97305499067921, -102.4938017818655 36.96990206863345, -102.4964379287525 36.96699352875253, -102.4993464686334 36.96435738186549, -102.5024993906792 36.96201901550789, -102.505866330527 36.96000094942606, -102.5094148627054 36.95832261869955, -102.5131108129098 36.95700018657071, -102.5169185871193 36.95604638878387, -102.5208015143868 36.95547041093311, -102.5247222 36.9552778, -102.5286428856132 36.95547041093311, -102.5325258128806 36.95604638878387, -102.5363335870902 36.95700018657071, -102.5400295372946 36.95832261869955, -102.543578069473 36.96000094942606, -102.5469450093208 36.96201901550789, -102.5500979313666 36.96435738186549, -102.5530064712475 36.96699352875253, -102.5556426181345 36.96990206863345, -102.5579809844921 36.973054990

In [56]:
print(lat_qfed[164], lon_qfed[59])
print(lat_qfed[44], lon_qfed[60])
print(type(fire_polygons.loc[81]))

47.45000076293945 -119.05000305175781
35.45000076293945 -118.94999694824219
<class 'pandas.core.series.Series'>
